In [4]:
# autoreload reloads modules automatically before entering the execution of code typed at the IPython prompt.
%reload_ext autoreload
%autoreload 2
%aimport bs_eda
%aimport bs_terminal

In [3]:
import bs_eda as beda
import bs_terminal as term

# Exploration des fichiers de données

In [5]:
all_data = beda.load_all_csv(exclude=["train_set_preprocessed.csv","train_set.csv","val_set.csv","test_set.csv"])

loading:['dataset/skoda.csv', 'dataset/focus.csv', 'dataset/unclean cclass.csv', 'dataset/cclass.csv', 'dataset/ford.csv', 'dataset/vw.csv', 'dataset/audi.csv', 'dataset/toyota.csv', 'dataset/bmw.csv', 'dataset/hyundi.csv', 'dataset/unclean focus.csv', 'dataset/merc.csv', 'dataset/vauxhall.csv']
Parsing dataset/skoda.csv
Parsing dataset/focus.csv
Parsing dataset/unclean cclass.csv
Parsing dataset/cclass.csv
Parsing dataset/ford.csv
Parsing dataset/vw.csv
Parsing dataset/audi.csv
Parsing dataset/toyota.csv
Parsing dataset/bmw.csv
Parsing dataset/hyundi.csv
Parsing dataset/unclean focus.csv
Parsing dataset/merc.csv
Parsing dataset/vauxhall.csv


### Structures des différents fichiers

In [1]:
def check_features(all_data):
    content = []

    features = {}
    columns = ['dataset', 'nbr features']
    data = []

    for key, dataset in all_data.items():
        feature_list = dataset.columns.tolist()
        data.append([key, str(len(feature_list))])

        for i in range(len(feature_list)):
            f = feature_list[i]
            if f in features:
                features[f] += 1
            else:
                features[f] = 1

    options = [{"justify": "right"}, {"justify": "left"}]
    content.append(term.create_table(
        title='Nombre de variable par jeu de données', columns=columns, data=data, options=options))
    content.append(term.create_section(
        title='Total des variables distinctes trouvées:', content=len(features.items())))

    data = []
    for key, value in features.items():
        data.append([key, value])
    columns = ['feature', 'total nbr']
    content.append(term.create_table(
        title='Occurence des différentes variables', columns=columns, data=data, options=options))

    term.article(
        title='Décompte des variables au travers des jeux de données', content=content)

In [11]:
check_features(all_data)

DÉCOMPTE DES VARIABLES AU TRAVERS DES JEUX DE DONNÉES ───────────────────────────────────────

Nombre de variable par jeu 
de données                 
┏━━━━━━━━━━┳━━━━━━━━━━━━━━┓
┃  dataset ┃ nbr features ┃
┡━━━━━━━━━━╇━━━━━━━━━━━━━━┩
│    skoda │ 9            │
│     ford │ 9            │
│       vw │ 9            │
│     audi │ 9            │
│   toyota │ 9            │
│      bmw │ 9            │
│   hyundi │ 9            │
│     merc │ 9            │
│ vauxhall │ 9            │
└──────────┴──────────────┘

Total des variables distinctes trouvées:
10

Occurence des différentes   
variables                   
┏━━━━━━━━━━━━━━┳━━━━━━━━━━━┓
┃      feature ┃ total nbr ┃
┡━━━━━━━━━━━━━━╇━━━━━━━━━━━┩
│        model │ 9         │
│         year │ 9         │
│        price │ 9         │
│ transmission │ 9         │
│      mileage │ 9         │
│    fuel_type │ 9         │
│          tax │ 8         │
│          mpg │ 9         │
│  engine_size │ 9         │
│       tax(£) │ 1         │
└──────────────┴───────────┘

Le dataset est composé de 13 fichiers. 

On y distingue 3 sortes de données:
1. 9 fichiers de marques: `["audi","bmw", "ford", "hyundi", "mercedes", "scoda", "toyota", "vauxhall", "vw"]`
1. 2 fichiers de modèles: `["cclass", "focus"]`
1. 2 fichiers de modèles *non nettoyés*: `["cclass", "focus"]`

Tous les csv du dataset n'ont pas le même nombre de features.

* Les fichiers par marque ont 9 colonnes
* Les fichiers par model ont 7 colonnes
* Les 2 fichiers *non nettoyés* ont 11 colonnes

In [ ]:
# removing 'unclean' data and by 'model' data
to_remove = ['unclean_focus','unclean_cclass','cclass','focus']

for i in range(len(to_remove)):
    all_data.pop(to_remove[i],None)

In [12]:
check_features(all_data)

DÉCOMPTE DES VARIABLES AU TRAVERS DES JEUX DE DONNÉES ───────────────────────────────────────

Nombre de variable par jeu 
de données                 
┏━━━━━━━━━━┳━━━━━━━━━━━━━━┓
┃  dataset ┃ nbr features ┃
┡━━━━━━━━━━╇━━━━━━━━━━━━━━┩
│    skoda │ 9            │
│     ford │ 9            │
│       vw │ 9            │
│     audi │ 9            │
│   toyota │ 9            │
│      bmw │ 9            │
│   hyundi │ 9            │
│     merc │ 9            │
│ vauxhall │ 9            │
└──────────┴──────────────┘

Total des variables distinctes trouvées:
10

Occurence des différentes   
variables                   
┏━━━━━━━━━━━━━━┳━━━━━━━━━━━┓
┃      feature ┃ total nbr ┃
┡━━━━━━━━━━━━━━╇━━━━━━━━━━━┩
│        model │ 9         │
│         year │ 9         │
│        price │ 9         │
│ transmission │ 9         │
│      mileage │ 9         │
│    fuel_type │ 9         │
│          tax │ 8         │
│          mpg │ 9         │
│  engine_size │ 9         │
│       tax(£) │ 1         │
└──────────────┴───────────┘

Mis à part les features `tax` et `tax(£)`

après exclusion des fichiers ci-dessus 

les données des différents fichiers pourront être fusionnées 

1. `model`: le model de la marque.
1. `year`: l'année d'enregistrement.
1. `price`: le prix attendu de la vente en £.
1. `transmission`: le type de transmission 
1. `mileage`: le nombre de miles parcouru.
1. `fuelType`: le type de carburant 
1. `tax`: taxe de mise en circulation.
1. `mpg`: le nombre de miles par gallon.  
1. `engineSize`: la taille du moteur en litres.